## Objectives
 - prepare data for dimesion reduction using PCA and clustering via K-Means
 - predict clusters using K-Means
 - create plots and data tables to present results 
 

In [317]:
#imports
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas
import plotly.express as px 
from path import Path

In [318]:
#load data
file_path = Path("../Resources/crypto_data.csv")
crypto_df = pd.read_csv(file_path)
crypto_df.head(5)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


- remove all currencies that aren't trading
- remove all currencies that don't have a defined algo
- remove the IsTrading column
- remove all currencies with at least one null value 
- remove all currencies without coins mined
- store the names of all currencies on a dataframe named coins_name, use cryptoDf.index as index 
- remove coinName column
- create dummies variables for all text features, store resulting data on dataframe named X
use standardScaler to standardize all data on X. do this before PCA and K-Means 

In [319]:
#remove all currencies that aren't trading
crypto_df = crypto_df[crypto_df["IsTrading"]==True]
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0


In [320]:
#drop IsTrading column
crypto_df = crypto_df.drop(["IsTrading"], axis=1)
#check for remaining null values
for column in crypto_df.columns:
    print(f"Column{column} has {crypto_df[column].isnull().sum()} null values")

ColumnUnnamed: 0 has 0 null values
ColumnCoinName has 0 null values
ColumnAlgorithm has 0 null values
ColumnProofType has 0 null values
ColumnTotalCoinsMined has 459 null values
ColumnTotalCoinSupply has 0 null values


In [321]:
#drop currencies with null values
crypto_df = crypto_df.dropna()
for column in crypto_df.columns:
    print(f"Column{column} has {crypto_df[column].isnull().sum()} null values")

ColumnUnnamed: 0 has 0 null values
ColumnCoinName has 0 null values
ColumnAlgorithm has 0 null values
ColumnProofType has 0 null values
ColumnTotalCoinsMined has 0 null values
ColumnTotalCoinSupply has 0 null values


In [322]:
crypto_df.head()

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [323]:
#reset index
crypto_df = crypto_df.set_index("Unnamed: 0")
crypto_df.index.name = None
crypto_df.head()

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000


In [324]:
#create coinName DF
coins_name = pd.DataFrame(crypto_df["CoinName"])
coins_name.head()

,CoinName
42,42 Coin
404,404Coin
808,808
1337,EliteCoin
BTC,Bitcoin


In [325]:
#drop coinName column from cryptoDf
crypto_df = crypto_df.drop(columns=["CoinName"], axis=1)
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
808,SHA-256,PoW/PoS,0.000000e+00,0
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000


In [326]:
#create dummie variables for algorithm and proof type
X = pd.get_dummies(crypto_df, columns=["Algorithm", "ProofType"])
X.head()

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
808,0.000000e+00,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [327]:
#standardized data with StandardScaler
X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:1])

[[-0.10282804 -0.03823841 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.05411338 -0.07664017 -0.03823596 -0.05411338 -0.05411338 -0.03823596
  -0.03823596 -0.18216065 -0.05411338 -0.03823596 -0.03823596 -0.08574929
  -0.03823596 -0.10160947 -0.06632365 -0.03823596 -0.03823596 -0.1642757
  -0.03823596 -0.03823596 -0.13908716 -0.03823596 -0.03823596 -0.07664017
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.06632365 -0.03823596
  -0.07664017 -0.08574929 -0.07664017 -0.03823596 -0.03823596 -0.12775161
  -0.1335313  -0.13908716 -0.03823596 -0.05411338 -0.03823596 -0.06632365
  -0.1689039  -0.03823596 -0.03823596 -0.03823596 -0.07664017 -0.17342199
  -0.33468341 -0.03823596 -0.08574929 -0.06632365 -0.05411338 -0.03823596
   1.42042992 -0.06632365 -0.03823596 -0.03823596 -0.06632365 -0.06632365
  -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596 -0.03823596
  -0.03823596 -0.41586681 -0.03823596 -0.19054822 -0.03823596 -0.10870529
  -0.07664017 -0.09400279 -0.03823596 -

use PCA to reduce dimensions

In [328]:
#initialized PCA model and get principal components
pca = PCA(n_components=3)
crypto_pca = pca.fit_transform(X_scaled)

In [329]:
#transform PCA data to dataframe
crypto_pca_df = pd.DataFrame(data=crypto_pca, columns=["PC 1", "PC 2", "PC 3"], index=crypto_df.index)
crypto_pca_df.head()

,PC 1,PC 2,PC 3
42,-0.233536,1.322211,-0.330667
404,-0.219232,1.320297,-0.330929
808,-0.171005,0.776626,-0.249281
1337,0.281443,1.940456,-0.423477
BTC,-0.215688,-1.502996,0.073623


- create an elbow curve to find best value for K using crypto_pca_df
- run K-Means algo to predict clusters for crypto data, using crypto_pca_df
- create a new dataframe named clustered_df that includes following columns: Algorithm, ProofType, TotalCoinsMined, TotalCoinSupply, PC 1, PC 2, PC 3, CoinName, Class

In [330]:
#create elbow curve
inertia = []
k = list(range(1,11))
#loop through k values and find inertia
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(crypto_pca_df)
    inertia.append(km.inertia_)
#use dataframe to plot elbow curve
elbow_data = {"k":k, "inertia": inertia}
elbow_df = pd.DataFrame(elbow_data)
elbow_df.hvplot.line(x="k", y="inertia", title="Elbow Curve", xticks=k)

:Curve   [k]   (inertia)

based on the elbow curve 4 clusters appears to be the best choice

In [331]:
#function to cluster and plot dataset
def testClusterAmount(df, clusters):
    df = df.copy()
    #initialize model
    model = KMeans(n_clusters=clusters, random_state=5)
    #fit the model
    model.fit(df)
    #make predictions
    predictions = model.predict(df)
    df["class"] = model.labels_
    return df

In [332]:
#use testClusterAmount for 4 clusters
clustered_df = testClusterAmount(crypto_pca_df, 4)
clustered_df.head()

,PC 1,PC 2,PC 3,class
42,-0.233536,1.322211,-0.330667,0
404,-0.219232,1.320297,-0.330929,0
808,-0.171005,0.776626,-0.249281,0
1337,0.281443,1.940456,-0.423477,0
BTC,-0.215688,-1.502996,0.073623,1


In [333]:
#merge dataframes
clustered_df = pd.concat([crypto_df, clustered_df, coins_name], axis=1)
clustered_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class,CoinName
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.233536,1.322211,-0.330667,0,42 Coin
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.219232,1.320297,-0.330929,0,404Coin
808,SHA-256,PoW/PoS,0.000000e+00,0,-0.171005,0.776626,-0.249281,0,808
1337,X13,PoW/PoS,2.927942e+10,314159265359,0.281443,1.940456,-0.423477,0,EliteCoin
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.215688,-1.502996,0.073623,1,Bitcoin


## visualizing the results

In [334]:
#plot in 3d
fig = px.scatter_3d(clustered_df, x="PC 1", y="PC 2", z="PC 3", color="class", symbol="class", width=800, hover_name="CoinName", hover_data=["Algorithm"])
fig.update_layout(legend=dict(x=0,y=1))
fig.show()

In [335]:
clustered_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinSupply,TotalCoinsMined,class]

In [336]:
#plot totalcoinsmined vs totalcoinsupply
fig = px.scatter(clustered_df, x="TotalCoinsMined", y="TotalCoinSupply", color="class", width=800, hover_name="CoinName", hover_data=["Algorithm"])
fig.show()

In [347]:
#remove valorbit and bittorrent 
clustered_df = clustered_df.drop("VAL")
clustered_df = clustered_df.drop("BTT")


In [349]:
#plot totalcoinsmined vs totalcoinsupply
fig = px.scatter(clustered_df, x="TotalCoinsMined", y="TotalCoinSupply", color="class", width=800, hover_name="CoinName", hover_data=["Algorithm"])
fig.show()